In [1]:
import os
import tensorflow as tf
import random

In [2]:
gtPath = "_conv3/"
rgbPath = "_out/"

gtFileNames = os.listdir(gtPath)
outFileNames = os.listdir(rgbPath)

matchingFiles = set(outFileNames) & set(gtFileNames)

In [3]:
NumTrainingImages = 10000
StartingIndex = 0

import random

FileNames = random.sample(matchingFiles, NumTrainingImages)

                            
gtFileNames = gtFileNames[StartingIndex:StartingIndex+NumTrainingImages]
outFileNames = outFileNames [StartingIndex:StartingIndex+NumTrainingImages]

for i in range(0,NumTrainingImages):
    gtFileNames[i] = gtPath + FileNames[i]
    outFileNames[i] = rgbPath + FileNames[i]
    

C:\Users\danie\AppData\Local\Temp\ipykernel_11404\233777074.py:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  FileNames = random.sample(matchingFiles, NumTrainingImages)


In [4]:
len(gtFileNames)

10000

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(outFileNames, gtFileNames, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [6]:
def loadImage(outPath, gtPath):
    
    img = tf.io.read_file(outPath)
    img = tf.image.decode_png(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    img = img[:1000, 460:1460, :] # Crop the photo (bonnet & make it square)
    img = tf.image.resize(img, (256, 256), method='nearest')

    mask = tf.io.read_file(gtPath)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.cast(mask, tf.float32) / 255.0
    mask = mask[:1000, 460:1460, :] # Crop the bonnet
    mask = tf.image.resize(mask, (256, 256), method='nearest')
    
    return img, mask

In [7]:
batchSize = 10
bufferSize = 500

def genDataset(outPath, gtPath, bufferSize, batchSize):
    
    imageList = tf.constant(outPath) 
    maskList = tf.constant(gtPath)
    dataset = tf.data.Dataset.from_tensor_slices((imageList, maskList))
    dataset = dataset.map(loadImage, num_parallel_calls=tf.data.AUTOTUNE)
    
    dataset = dataset.cache().shuffle(bufferSize).batch(batchSize)
    
    return dataset

In [8]:
trainDataset = genDataset(X_train, y_train, bufferSize, batchSize)
valDataset = genDataset(X_val, y_val, bufferSize, batchSize)
testDataset = genDataset(X_test, y_test, bufferSize, batchSize)

In [10]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from kerastuner import HyperModel, Hyperband


def encoderBlock(inputs, filters, maxPool=True):
    
    X = Conv2D(filters, 3, padding="same")(inputs)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)

    X = Conv2D(filters, 3, padding="same")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)

    skip = X
    
    if maxPool:
        X = MaxPooling2D(pool_size=(2, 2))(X)        

    return X, skip

C:\Users\danie\AppData\Local\Temp\ipykernel_11404\2492856641.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel, Hyperband


In [11]:
def decoderBlock(inputs, skip, filters):
    
    X = Conv2DTranspose(filters, 3, strides=(2,2), padding="same")(inputs) 
    
    X = concatenate([X, skip], axis=3) #concat prev skip connection

    X = Conv2D(filters, 3, padding="same")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    
    X = Conv2D(filters, 3, padding="same")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    
  
    return X

In [12]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import HyperModel


class UNetHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        inputs = Input(self.input_shape)
        
        # Define hyperparameters to tune
        filters = hp.Choice('filters', [32, 64, 128])
        
        # Encoder
        X1, S1 = encoderBlock(inputs, filters, maxPool=True)
        X2, S2 = encoderBlock(X1, filters * 2, maxPool=True)
        X3, S3 = encoderBlock(X2, filters * 4, maxPool=True)
        X4, S4 = encoderBlock(X3, filters * 8, maxPool=True)
        X5, S5 = encoderBlock(X4, filters * 16, maxPool=False)
        
        # Decoder
        X6 = decoderBlock(X5, S4, filters * 8)
        X7 = decoderBlock(X6, S3,  filters * 4)
        X8 = decoderBlock(X7, S2,  filters = filters * 2)
        X9 = decoderBlock(X8, S1,  filters = filters)
        
        X10 = Conv2D(filters, 3,activation='relu', padding='same')(X9)
        X11 = Conv2D(filters=self.num_classes, kernel_size=(1, 1), activation='sigmoid', padding='same')(X10)
        
        modelUnet = Model(inputs=inputs, outputs=X11)
        modelUnet.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
        modelUnet.summary()
        return modelUnet

In [13]:
imgShape = (256,256,3)
labels = 2 # True/False

hypermodel = UNetHyperModel(input_shape=imgShape, num_classes=labels)

tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=30,
    directory='keras_tuner_dir',
    project_name='unet_hyperband'
)


Reloading Tuner from keras_tuner_dir\unet_hyperband\tuner0.json


In [14]:
# Run the hyperparameter search
tuner.search(trainDataset, validation_data=valDataset)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps}")

Trial 3 Complete [00h 16m 50s]
val_accuracy: 0.9869365692138672

Best val_accuracy So Far: 0.9869365692138672
Total elapsed time: 02h 03m 45s
Best hyperparameters: <keras_tuner.src.engine.hyperparameters.hyperparameters.HyperParameters object at 0x000001B619842790>


In [19]:
best_hps.get('filters')


64